In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd

import kraft

In [ ]:
w = pd.read_csv("w.tsv", sep="\t", index_col=0)

w.columns.name = "Factor"

print(w.shape)

h = pd.read_csv("h.tsv", sep="\t", index_col=0)

h.columns.name = "Sample"

print(h.shape)

In [ ]:
gps_map = kraft.GPSMap(w=w.T, h=h)

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.plot(w_or_h)

In [ ]:
for w_or_h, elements, element_x_dimension in (
    ("w", gps_map.w_elements, gps_map.w_element_x_dimension),
    ("h", gps_map.h_elements, gps_map.h_element_x_dimension),
):

    element_value = pd.Series(
        np.apply_along_axis(np.sum, 1, element_x_dimension),
        index=elements,
        name="Element Value",
    )

    element_value[::3] = np.nan

    gps_map.plot(w_or_h, element_value=element_value)

In [ ]:
for w_or_h, elements, node_x_dimension in (
    ("w", gps_map.w_elements, gps_map.w),
    ("h", gps_map.h_elements, gps_map.h),
):

    element_label = pd.Series(
        node_x_dimension.argmax(axis=0), index=elements, name="Element Label"
    )

    gps_map.set_element_label(w_or_h, element_label)

    gps_map.plot(w_or_h)

In [ ]:
for w_or_h, elements, element_x_dimension in (
    ("w", gps_map.w_elements, gps_map.w_element_x_dimension),
    ("h", gps_map.h_elements, gps_map.h_element_x_dimension),
):

    element_value = pd.Series(
        np.apply_along_axis(np.sum, 1, element_x_dimension),
        index=elements,
        name="Element Value",
    )

    gps_map.plot(w_or_h, element_value=element_value)

In [ ]:
for w_or_h, node_x_element in (("w", w.T), ("h", h)):

    elemenet_x_predicting_element = node_x_element.iloc[:, :10]

    elemenet_x_predicting_element.columns.name = "New {}".format(
        node_x_element.columns.name
    )

    gps_map.predict(w_or_h, elemenet_x_predicting_element)

In [ ]:
h = pd.read_csv("kras_gps_map.h.tsv", sep="\t", index_col=0)

h.index.name = "Component"

h.columns.name = "Cell Line"

h = h.apply(kraft.normalize_series, axis=1, args=("-0-",)).clip(lower=-3, upper=3)

h = h.apply(kraft.normalize_series, axis=1, args=("0-1",))

gps_map = kraft.GPSMap(h=h, h_pull_power=2.4)

element_marker_size = 12

In [ ]:
gps_map.set_element_label(
    "h", pd.read_csv("kras_gps_map.k_x_h_element.tsv", sep="\t", index_col=0).loc["K15"]
)

gps_map.plot("h", element_marker_size=element_marker_size)

In [ ]:
h_ = h.copy()

h_.columns.name = "New {}".format(h.columns.name)

gps_map.predict("h", h_, element_marker_size=element_marker_size)

In [ ]:
gps_map_path = "gps_map.pickle.gz"

kraft.write_gps_map(gps_map, gps_map_path)

In [ ]:
gps_map = kraft.read_gps_map(gps_map_path)

gps_map.plot("h", element_marker_size=element_marker_size)